# [TUTORIAL] Constrained Task Planning (Sweep)

This tutorial is written to instruct basic usage of the task & motion planning pipeline  
One Indy7 robot and several environment geometries will be added and floor-wiping task will be conducted.  
Here, the wiping task is defined as 1) contact with floor, 2) starting waypoint, 3) goal waypoint.  
Thus, any motion that satisfies the constraint will be generated; it may not look like real wiping.  

## set running directory to project source

In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

## init combined robot config

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7, ((0,0,0), (0,0,0)),
                INDY_IP)]
              , connection_list=[False])

connection command:
indy0: False


## create scene builder

In [3]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# s_builder.reset_reference_coord(ref_name="floor")
gscene = s_builder.create_gscene(crob)

Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.


Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]


## init planning pipeline

In [4]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


```
open web ui on <your ip>:8050
click geometry items / Handles / Binders to highlight geometry on RVIZ
other functions may be buggy.. please report
```

## add environment

In [5]:
from pkg.geometry.geometry import *

floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (0.3,0.5,0.01), (0.5,0,-0.005), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=False)
floor_col = gscene.create_safe(GEOTYPE.BOX, "floor_col", "base_link", (0.3,0.5,0.01), (0.5,0,-0.006), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=False, fixed=True, collision=True)
wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (-0.2,0,0), 
                           rpy=(0,np.pi/2,), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
goal = gscene.create_safe(GEOTYPE.BOX, "goal", "base_link", (0.1,0.1,0.01), (0.3,-0.4,-0.005), 
                          rpy=(0,0,0), color=(0.8,0.8,0.2,1), display=True, fixed=True, collision=False)
goal_col = gscene.create_safe(GEOTYPE.BOX, "goal_col", "base_link", (0.1,0.1,0.01), (0.3,-0.4,-0.006), 
                          rpy=(0,0,0), color=(0.8,0.8,0.2,1), display=False, fixed=True, collision=True)

Please create a subscriber to the marker
   Use a production WSGI server instead.
 * Debug mode: off


In [6]:
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True, exclude_link=["panda1_link7"])

In [7]:
from pkg.utils.code_scraps import add_indy_gripper_asm2
add_indy_gripper_asm2(gscene, "indy0", "indy0_tcp")

## add wp

In [8]:
wp11 = gscene.create_safe(GEOTYPE.BOX, "wp11", "base_link", (0.08,0.08,0.01), (0.6,0.2,-0.005),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
wp12 = gscene.create_safe(GEOTYPE.BOX, "wp12", "base_link", (0.08,0.08,0.01), (0.6,-0.2,-0.005), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
line1 = gscene.create_safe(GEOTYPE.BOX, "wline1", "base_link", (0.01,0.5,1e-6), (0.6,0,0), rpy=(0,np.pi/2,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)

wp21 = gscene.create_safe(GEOTYPE.BOX, "wp21", "base_link", (0.08,0.08,0.01), (0.5,0.2,-0.005),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
wp22 = gscene.create_safe(GEOTYPE.BOX, "wp22", "base_link", (0.08,0.08,0.01), (0.5,-0.2,-0.005), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
line2 = gscene.create_safe(GEOTYPE.BOX, "wline2", "base_link", (0.01,0.5,1e-6), (0.5,0,0), rpy=(0,np.pi/2,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)

wp31 = gscene.create_safe(GEOTYPE.BOX, "wp31", "base_link", (0.08,0.08,0.01), (0.4,0.2,-0.005),rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
wp32 = gscene.create_safe(GEOTYPE.BOX, "wp32", "base_link", (0.08,0.08,0.01), (0.4,-0.2,-0.005), rpy=(0,0,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)
line3 = gscene.create_safe(GEOTYPE.BOX, "wline3", "base_link", (0.01,0.5,1e-6), (0.4,0,0), rpy=(0,np.pi/2,0), 
                         color=(0.8,0.2,0.2,0.2), display=True, fixed=True, collision=False)

## add brush

In [9]:
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_body", link_name="base_link", dims=(0.1,0.07,0.02), 
                   center=(0.3,-0.4,0.04), rpy=(0,0,0), color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=False)
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_handle", link_name="base_link", dims=(0.1,0.03,0.05), center=(0,0,0.035), rpy=(0,0,0), 
                   color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=False, parent="brush_body")
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_face_col", link_name="base_link", dims=(0.1,0.06,0.028), center=(0,0,-0.025), rpy=(0,0,0), 
                   color=(0.8,0.8,0.8,1), display=True, collision=False, fixed=False, parent="brush_body")
gscene.create_safe(gtype=GEOTYPE.BOX, name="brush_face", link_name="base_link", dims=(0.1,0.06,0.03), center=(0,0,-0.025), rpy=(0,0,0), 
                   color=(0.8,0.8,0.8,1), display=True, collision=False, fixed=False, parent="brush_body")

## add box

In [10]:
gbox1 = gscene.create_safe(gtype=GEOTYPE.BOX, name="box1", link_name="base_link", dims=(0.05,0.05,0.05), 
                   center=(0.5,0.1,0.025), rpy=(0,0,0), color=(0.7,0.3,0.3,1), display=True, collision=True, fixed=False)

gbox2 = gscene.create_safe(gtype=GEOTYPE.BOX, name="box2", link_name="base_link", dims=(0.05,0.05,0.05), 
                   center=(0.4,-0.1,0.025), rpy=(0,0,0), color=(0.7,0.3,0.3,1), display=True, collision=True, fixed=False)

## Register binders

In [11]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepTool

In [12]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="indy0_tcp", 
                 dims=(0.01,)*3, center=(0,0,0.14), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)

In [13]:
pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))
pscene.create_binder(bname="floor", gname="floor", _type=PlacePlane)
pscene.create_binder(bname="goal_bd", gname="goal", _type=PlacePlane, point=(0,0,0.005), rpy=(0,0,0))
pscene.create_binder(bname="brush_face", gname="brush_face", _type=SweepTool, point=(0,0,-0.015), rpy=(0,0,0))

## add objects

In [14]:
from pkg.planning.constraint.constraint_subject import CustomObject, Grasp2Point, PlacePoint, SweepPoint, SweepTask, BoxObject

In [15]:
brush_handle = gscene.NAME_DICT["brush_handle"]
brush_face = gscene.NAME_DICT["brush_face"]
brush = pscene.create_subject(oname="brush", gname="brush_body", _type=CustomObject, 
                             action_points_dict = {"handle": Grasp2Point("handle", brush_handle, [0,0,0], [np.pi/2,0,0]),
                                                   "face": PlacePoint("face", brush_face, [0,0,-0.015], [0,0,0])})

In [16]:
box1 = pscene.create_subject(oname="box1", gname="box1", _type=BoxObject, hexahedral=True)
box2 = pscene.create_subject(oname="box2", gname="box2", _type=BoxObject, hexahedral=True)

In [17]:
from pkg.planning.constraint.constraint_common import MotionConstraint
from pkg.planning.constraint.constraint_subject import AbstractTask
from pkg.planning.constraint.constraint_subject import SweepLineTask

In [18]:
sweep1 = pscene.create_subject(oname="sweep1", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp11": SweepPoint("wp11", wp11, [0,0,0.005], [0,0,0]),
                                                   "wp12": SweepPoint("wp12", wp12, [0,0,0.005], [0,0,0])},
                            geometry_vertical = line1)
sweep2 = pscene.create_subject(oname="sweep2", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp21": SweepPoint("wp21", wp21, [0,0,0.005], [0,0,0]),
                                                   "wp22": SweepPoint("wp22", wp22, [0,0,0.005], [0,0,0])},
                            geometry_vertical = line2)
sweep3 = pscene.create_subject(oname="sweep3", gname="floor", _type=SweepLineTask, 
                             action_points_dict = {"wp31": SweepPoint("wp31", wp31, [0,0,0.005], [0,0,0]),
                                                   "wp32": SweepPoint("wp32", wp32, [0,0,0.005], [0,0,0])},
                            geometry_vertical = line3)

### planners

In [19]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
mplan.update_gscene()
from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

## motion filters

In [21]:
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
from pkg.planning.filtering.latticized_filter import LatticedChecker
TOOL_LINK = "indy0_tcp"
TOOL_LINK_BUNDLE = ["indy0_tcp", "indy0_link6"]

gcheck = GraspChecker(pscene)
rcheck = ReachChecker(pscene)
checkers_all = [gcheck, rcheck]
# lcheck = LatticedChecker(pscene, gcheck)
# checkers_all.append(lcheck)

In [22]:
mplan.motion_filters = checkers_all

## Test plan

In [23]:
from pkg.planning.constraint.constraint_common \
            import sample_redundancy, combine_redundancy
gtimer = GlobalTimer.instance()
initial_state = pscene.initialize_state(crob.home_pose)

# Node Sampler

In [26]:
from pkg.planning.sampling.node_sampling import UniformNodeSampler

In [28]:
tplan.new_node_sampler = UniformNodeSampler(0.5)
tplan.parent_node_sampler = UniformNodeSampler(0.5)

In [29]:
gtimer.reset()
goal_nodes = [("floor", "floor", "goal", 2, 2, 2)]
gtimer.tic("plan")
ppline.search(initial_state, goal_nodes, verbose=True, display=False, dt_vis=0.01, timeout_loop=300, multiprocess=True, timeout=1, timeout_constrained=1)
gtimer.toc("plan")
schedules = ppline.tplan.find_schedules()
schedules_sorted = ppline.tplan.sort_schedule(schedules)
snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])

Use 20/20 agents
try: 0 - ('floor', 'floor', 'floor', 0, 0, 0)->('floor', 'floor', 'floor', 1, 0, 0)
result: 0 - ('floor', 'floor', 'floor', 0, 0, 0)->('floor', 'floor', 'floor', 1, 0, 0) = fail
try: 0 - ('floor', 'floor', 'floor', 0, 0, 0)->('grip0', 'floor', 'floor', 0, 0, 0)
result: 0 - ('floor', 'floor', 'floor', 0, 0, 0)->('grip0', 'floor', 'floor', 0, 0, 0) = fail
try: 0 - ('floor', 'floor', 'floor', 0, 0, 0)->('floor', 'floor', 'floor', 1, 0, 0)
result: 0 - ('floor', 'floor', 'floor', 0, 0, 0)->('floor', 'floor', 'floor', 1, 0, 0) = fail
try: 0 - ('floor', 'floor', 'floor', 0, 0, 0)->('floor', 'floor', 'grip0', 0, 0, 0)
try transition motion
transition motion tried: True
result: 0 - ('floor', 'floor', 'floor', 0, 0, 0)->('floor', 'floor', 'grip0', 0, 0, 0) = success
branching: 0->1 (0.15/300.0 s, steps/err: 35(45.0270175934 ms)/0.00167462030886)
try: 1 - ('floor', 'floor', 'grip0', 0, 0, 0)->('floor', 'floor', 'goal', 0, 0, 0)
try transition motion
transition motion tried: True


try: 4 - ('floor', 'floor', 'grip0', 0, 1, 0)->('floor', 'floor', 'grip0', 0, 2, 0)
result: 4 - ('floor', 'floor', 'grip0', 0, 1, 0)->('floor', 'floor', 'grip0', 0, 2, 0) = fail
try: 6 - ('floor', 'floor', 'floor', 0, 0, 0)->('floor', 'floor', 'floor', 0, 1, 0)
try transition motion
transition motion tried: True
try: 2 - ('floor', 'floor', 'goal', 0, 0, 0)->('floor', 'grip0', 'goal', 0, 0, 0)
result: 5 - ('floor', 'floor', 'grip0', 0, 1, 0)->('floor', 'floor', 'grip0', 0, 2, 0) = fail
try transition motion
transition motion tried: True
result: 1 - ('floor', 'floor', 'grip0', 0, 0, 0)->('floor', 'floor', 'floor', 0, 0, 0) = success
try: 2 - ('floor', 'floor', 'goal', 0, 0, 0)->('grip0', 'floor', 'goal', 0, 0, 0)
try: 5 - ('floor', 'floor', 'grip0', 0, 1, 0)->('floor', 'floor', 'grip0', 0, 2, 0)
result: 7 - ('floor', 'floor', 'grip0', 0, 0, 0)->('floor', 'floor', 'grip0', 1, 0, 0) = success
result: 1 - ('floor', 'floor', 'grip0', 0, 0, 0)->('floor', 'floor', 'grip0', 0, 1, 0) = success
r

try: 1 - ('floor', 'floor', 'grip0', 0, 0, 0)->('floor', 'floor', 'grip0', 0, 0, 1)
result: 14 - ('floor', 'grip0', 'goal', 0, 0, 0)->('floor', 'floor', 'goal', 0, 0, 0) = fail
try: 14 - ('floor', 'grip0', 'goal', 0, 0, 0)->('floor', 'goal', 'goal', 0, 0, 0)
result: 2 - ('floor', 'floor', 'goal', 0, 0, 0)->('floor', 'floor', 'goal', 0, 0, 1) = fail
try: 1 - ('floor', 'floor', 'grip0', 0, 0, 0)->('floor', 'floor', 'grip0', 0, 0, 1)
result: 2 - ('floor', 'floor', 'goal', 0, 0, 0)->('floor', 'floor', 'goal', 0, 0, 1) = fail
try: 14 - ('floor', 'grip0', 'goal', 0, 0, 0)->('floor', 'goal', 'goal', 0, 0, 0)
try: 3 - ('floor', 'floor', 'floor', 0, 0, 0)->('floor', 'floor', 'floor', 0, 0, 1)
result: 2 - ('floor', 'floor', 'goal', 0, 0, 0)->('floor', 'grip0', 'goal', 0, 0, 0) = success
try: 16 - ('floor', 'grip0', 'goal', 0, 0, 0)->('floor', 'grip0', 'goal', 0, 1, 0)
try transition motion
try: 7 - ('floor', 'floor', 'grip0', 0, 0, 0)->('floor', 'floor', 'grip0', 1, 0, 0)
try: 9 - ('floor', 'flo

try: 8 - ('floor', 'floor', 'floor', 0, 0, 0)->('grip0', 'floor', 'floor', 0, 0, 0)
result: 16 - ('floor', 'grip0', 'goal', 0, 0, 0)->('floor', 'grip0', 'goal', 1, 0, 0) = fail
try: 21 - ('floor', 'floor', 'grip0', 0, 0, 1)->('floor', 'floor', 'grip0', 0, 0, 2)
result: 8 - ('floor', 'floor', 'floor', 0, 0, 0)->('grip0', 'floor', 'floor', 0, 0, 0) = fail
try transition motion
branching: 1->21 (1.0/300.0 s, steps/err: 32(66.8578147888 ms)/0.00165712498788)
result: 19 - ('floor', 'floor', 'grip0', 0, 0, 1)->('floor', 'floor', 'grip0', 0, 0, 2) = fail
try: 22 - ('floor', 'floor', 'grip0', 0, 0, 1)->('floor', 'floor', 'grip0', 0, 0, 2)
result: 1 - ('floor', 'floor', 'grip0', 0, 0, 0)->('floor', 'floor', 'grip0', 0, 0, 1) = success
result: 21 - ('floor', 'floor', 'grip0', 0, 0, 1)->('floor', 'floor', 'grip0', 0, 0, 2) = fail
try: 22 - ('floor', 'floor', 'grip0', 0, 0, 1)->('floor', 'floor', 'grip0', 0, 0, 2)
branching: 1->22 (1.01/300.0 s, steps/err: 32(246.911048889 ms)/0.00111379289648)
tr

try: 27 - ('grip0', 'floor', 'floor', 0, 0, 0)->('grip0', 'floor', 'floor', 1, 0, 0)
result: 26 - ('floor', 'floor', 'grip0', 1, 0, 0)->('floor', 'floor', 'grip0', 2, 0, 0) = fail
result: 27 - ('grip0', 'floor', 'floor', 0, 0, 0)->('grip0', 'floor', 'floor', 1, 0, 0) = fail
result: 27 - ('grip0', 'floor', 'floor', 0, 0, 0)->('goal', 'floor', 'floor', 0, 0, 0) = fail
try: 28 - ('grip0', 'floor', 'goal', 0, 0, 0)->('grip0', 'floor', 'goal', 0, 1, 0)
try: 28 - ('grip0', 'floor', 'goal', 0, 0, 0)->('grip0', 'floor', 'goal', 0, 1, 0)
try: 0 - ('floor', 'floor', 'floor', 0, 0, 0)->('floor', 'floor', 'grip0', 0, 0, 0)
result: 28 - ('grip0', 'floor', 'goal', 0, 0, 0)->('grip0', 'floor', 'goal', 1, 0, 0) = fail
try: 2 - ('floor', 'floor', 'goal', 0, 0, 0)->('floor', 'floor', 'goal', 1, 0, 0)
result: 27 - ('grip0', 'floor', 'floor', 0, 0, 0)->('grip0', 'floor', 'floor', 0, 1, 0) = fail
result: 2 - ('floor', 'floor', 'goal', 0, 0, 0)->('floor', 'floor', 'goal', 1, 0, 0) = fail
transition motion t

...

try transition motion
result: 2233 - ('floor', 'grip0', 'floor', 2, 2, 2)->('floor', 'goal', 'floor', 2, 2, 2) = success
branching: 2251->2260 (121.2/300.0 s, steps/err: 28(58.9098930359 ms)/0.00145549711461)
try joint motion
result: 2251 - ('floor', 'floor', 'grip0', 2, 2, 2)->('floor', 'floor', 'goal', 2, 2, 2) = success
try: 2261 - ('floor', 'goal', 'floor', 2, 2, 2)->('floor', 'grip0', 'floor', 2, 2, 2)
branching: 2233->2261 (121.21/300.0 s, steps/err: 25(474.500894547 ms)/0.00173724147988)
branching: 2251->2262 (121.21/300.0 s, steps/err: 37(166.375875473 ms)/0.00123841019556)
constrained motion tried: False
result: 2258 - ('floor', 'floor', 'goal', 2, 2, 2)->('grip0', 'floor', 'goal', 2, 2, 2) = fail
result: 2240 - ('floor', 'grip0', 'floor', 2, 2, 2)->('floor', 'floor', 'floor', 2, 2, 2) = success
try: 2263 - ('floor', 'floor', 'floor', 2, 2, 2)->('floor', 'floor', 'grip0', 2, 2, 2)
branching: 2240->2263 (121.22/300.0 s, steps/err: 52(748.337984085 ms)/0.00151192385881)
transiti

++ adding return motion to acquired answer ++
++ adding return motion to acquired answer ++
branching: 2251->2276 (121.5/300.0 s, steps/err: 37(376.578092575 ms)/0.00199351807916)
try joint motion
try: 2260 - ('floor', 'floor', 'goal', 2, 2, 2)->('floor', 'grip0', 'goal', 2, 2, 2)
try: 2264 - ('floor', 'floor', 'goal', 2, 2, 2)->('floor', 'grip0', 'goal', 2, 2, 2)
======================= terminated 9: first answer acquired ===============================
result: 2260 - ('floor', 'floor', 'goal', 2, 2, 2)->('floor', 'grip0', 'goal', 2, 2, 2) = fail
try joint motion
++ adding return motion to acquired answer ++
======================= terminated 15: first answer acquired ===============================
try joint motion
try transition motion
======================= terminated 3: first answer acquired from other agent ===============================
joint motion tried: True
try: 2262 - ('floor', 'floor', 'goal', 2, 2, 2)->('floor', 'grip0', 'goal', 2, 2, 2)
transition motion tried: True
re

AttributeError: PlanningPipeline instance has no attribute 'sort_schedule'

transition motion tried: False
result: 2249 - ('floor', 'floor', 'floor', 2, 2, 2)->('grip0', 'floor', 'floor', 2, 2, 2) = fail
======================= terminated 10: first answer acquired from other agent ===============================
constrained motion tried: False
result: 2213 - ('floor', 'floor', 'grip0', 2, 1, 2)->('floor', 'floor', 'grip0', 2, 2, 2) = fail
======================= terminated 17: first answer acquired from other agent ===============================


Process Process-18:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
    self.run()
Process Process-15:
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
Traceback (most recent call last):
Process Process-2:
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
Traceback (most recent call last):
    self.run()
  File "/usr/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "pkg/planning/pipeline.py", line 152, in __search_loop
    display=display, dt_vis=dt_vis, **kwargs)
  File "pkg/planning/pipeline.py", line 200, in test_connection
    self.mplan.plan_transition(from_state, to_state, redundancy_dict=redundancy_dict, **kwargs)
  File "pkg/planning/motion/interface.py", line 105, in plan

In [31]:
print(gtimer)

plan: 	121583.0 ms/1 = 121583.464 ms (121583.464/121583.464)



## play searched plan

In [32]:
ppline.play_schedule(snode_schedule[:2], period=0.01)
pscene.set_object_state(initial_state)
gscene.show_pose(crob.home_pose)
time.sleep(0.5)
gscene.show_pose(crob.home_pose)

In [34]:
ppline.play_schedule(snode_schedule, period=0.03)

In [30]:
len(snode_schedule)

20

In [31]:
for i_s,  snode in enumerate(snode_schedule):
    print("{}: {}".format(i_s, snode.state.node))

0: ('floor', 'floor', 0, 0, 0)
1: ('floor', 'grip0', 0, 0, 0)
2: ('floor', 'goal', 0, 0, 0)
3: ('floor', 'grip0', 0, 0, 0)
4: ('floor', 'floor', 0, 0, 0)
5: ('grip0', 'floor', 0, 0, 0)
6: ('goal', 'floor', 0, 0, 0)
7: ('goal', 'grip0', 0, 0, 0)
8: ('goal', 'grip0', 0, 1, 0)
9: ('goal', 'grip0', 0, 2, 0)
10: ('goal', 'grip0', 0, 2, 1)
11: ('goal', 'grip0', 0, 2, 2)
12: ('goal', 'floor', 0, 2, 2)
13: ('grip0', 'floor', 0, 2, 2)
14: ('floor', 'floor', 0, 2, 2)
15: ('floor', 'grip0', 0, 2, 2)
16: ('floor', 'grip0', 1, 2, 2)
17: ('floor', 'grip0', 2, 2, 2)
18: ('floor', 'goal', 2, 2, 2)
19: ('floor', 'goal', 2, 2, 2)


## NOTE
* 위에 가운데 열 어떻게  0,1,1,1,1,2가 나오지? -> 일방통행 작업 플래그 추가
* 균등 샘플링 - 샘플 할때마다 노드별/전환별 확률 조정
* goal-directed extension 추가.

In [56]:
tplan.node_dict[('grip0', 'goal', 0, 1, 2)]

{('floor', 'goal', 0, 1, 2),
 ('goal', 'goal', 0, 1, 2),
 ('grip0', 'goal', 0, 2, 2),
 ('grip0', 'goal', 1, 1, 2)}

## extend preserving goal-matching items

In [25]:
print(gtimer)

plan: 	207291.0 ms/10 = 20729.062 ms (9041.162/45383.357)



## extend only no reservation

In [25]:
print(gtimer)

plan: 	270954.0 ms/10 = 27095.38 ms (6255.591/68399.08)



## no extension

In [25]:
print(gtimer)

plan: 	226519.0 ms/10 = 22651.91 ms (8120.631/69353.952)



## extend_toward goal

In [30]:
print(gtimer)

plan: 	204994.0 ms/10 = 20499.448 ms (8105.979/60408.919)

